In [1]:
import numpy as np
from scipy.misc import imread
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from functions1 import *

def surface_plot(surface,title):
    M,N = surface.shape

    ax_rows = np.arange(M)
    ax_cols = np.arange(N)

    [X,Y] = np.meshgrid(ax_cols, ax_rows)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(X,Y,surface,cmap=cm.viridis,linewidth=0)
    plt.title(title)

if __name__ == '__main__':

    # Load the terrain
    terrain1 = imread('SRTM_data_Norway_1.tif')

    #terrain2 = imread('SRTM_data_Norway_2.tif')

    # Show the terrain
    plt.figure()
    plt.title('Terrain over Norway 1')
    plt.imshow(terrain1, cmap='gray')
    plt.xlabel('X')
    plt.ylabel('Y')

    surface_plot(terrain1,'Surface plot over Norway 1')

    #plt.show()

    # Extract a smaller patch of the terrain
    row_start = 1950
    row_end = 2050

    col_start = 1200
    col_end = 1450

    terrain1_patch = terrain1[row_start:row_end, col_start:col_end]

    surface_plot(terrain1_patch,'Surface plot ovar a patch of Norway 1')

    #plt.show()

    # Perform a OLS fitting using the whole axis pf the terrain data
    num_rows, num_cols = terrain1_patch.shape

    rows = np.arange(num_rows)
    cols = np.arange(num_cols)

    [C,R] = np.meshgrid(cols,rows)

    x = C.reshape(-1,1)
    y = R.reshape(-1,1)
    z = terrain1_patch.flatten()

    data = np.c_[np.ones((num_rows*num_cols,1)), x, y, x**2, x*y, y**2, \
                x**3, x**2*y, x*y**2, y**3, \
                x**4, x**3*y, x**2*y**2, x*y**3,y**4, \
                x**5, x**4*y, x**3*y**2, x**2*y**3,x*y**4, y**5]

    beta_ls = np.linalg.inv(data.T @ data) @ data.T @ z

    fitted_terrain = np.zeros((num_rows, num_cols))
    for i in range(num_rows):
        y_ = rows[i]
        for j in range(num_cols):
            x_ = cols[j]

            data_vec = np.array([1, x_, y_, x_**2, x_*y_, y_**2, \
                                x_**3, x_**2*y_, x_*y_**2, y_**3, \
                                x_**4, x_**3*y_, x_**2*y_**2, x_*y_**3,y_**4, \
                                x_**5, x_**4*y_, x_**3*y_**2, x_**2*y_**3,x_*y_**4,y_**5])
            fitted_terrain[i,j] = data_vec @ beta_ls

    surface_plot(fitted_terrain,'Fitted terrain surface')

    #plt.show()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [5]:
xVector=x
print(np.size(x))
yVector=y
print(np.size(y))
zVector=z
#matrixSizey=np.size(y1,1)
#matrixSizex=np.size(x1,0)

25000
25000


In [ ]:
#Preparation for k-folds CV
sizeVector=np.size(zVector)
indeces=np.linspace(0,sizeVector-1,sizeVector)
indeces=indeces.astype(int)
numberOfFolds=10
folds = k_folds_CV(indeces, numberOfFolds)

In [ ]:
polynom_oders=[2,3,4,5]
zPredictmatrix,statsMatrix,betaMatrix=olsModel(polynom_oders,xVector,yVector,zVector,numberOfFolds,folds,indeces)

In [ ]:
print(beta_ls)

In [ ]:
print(data_vec)

In [ ]:
print(data_vec @ beta_ls)

In [ ]:
fitted_terrain2 = data @ beta_ls
fitted_terrain2 = fitted_terrain2.reshape(100,250)
surface_plot(fitted_terrain2,'Fitted terrain surface')

In [ ]:
z_predict=fitted_terrain.flatten()

In [ ]:
print(MSE(z,z_predict))

In [ ]:
plt.figure()
plt.title('Terrain over Norway 1')
plt.imshow(fitted_terrain, cmap='gray')
plt.xlabel('X')
plt.ylabel('Y')

In [ ]:
plt.figure()
plt.title('Terrain over Norway 1')
plt.imshow(fitted_terrain, cmap='gray')
plt.xlabel('X')
plt.ylabel('Y')

In [ ]:
beta1,XXinv1 =OSLregression(xVector,yVector,zVector,5)

In [ ]:
lambda_values=[1e-3,1e-2,1e-1,1,10,1e2]
polynom_oders=[2,3,4,5]
zPredictmatrix,statsMatrix,betaMatrix=ridge_regress(lambda_values,polynom_oders,xVector,yVector,zVector,numberOfFolds,folds,indeces)
for j,order in enumerate(polynom_oders):
    for  h,lbd in enumerate(lambda_values): 
        print(lbd)
        print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,h,j])))
        print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,h,j])))
        print('\n')

In [ ]:
matrixSizey=np.size(C,1)
matrixSizex=np.size(R,0)
beta1=np.mean(betaMatrix[:,:,0,3],1)
z_pred=computeZpredict(xVector,yVector,beta1,5)
bias,var=computeBiasandVar(zPredictmatrix[:,:,0,3],zVector)
print('Bias for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))
zPredictReshape=np.reshape(z_pred,(matrixSizex,matrixSizey))
fig1=plotTheSurface(R,C,zPredictReshape)
fig1.show()

print(str(MSE(z_pred,zVector)))
print(bias + var)

In [ ]:
plt.figure()
plt.title('Terrain over Norway 1')
plt.imshow(zPredictReshape, cmap='gray')
plt.xlabel('X')
plt.ylabel('Y')

In [6]:
sizeVector=6480000
indeces=np.linspace(0,sizeVector-1,sizeVector)
indeces=indeces.astype(int)
numberOfFolds=900
folds = k_folds_CV(indeces, numberOfFolds)

KeyboardInterrupt: 